In [1]:
from IslandTime import retrieve_island_info, PreTimeSeries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import folium
import webbrowser
import shapely

import osmnx as ox
%load_ext autoreload
%autoreload 2

In [78]:
for island in ['Olhugiri (South Maalhosmadulu)']:
    west, north, east, south = 5.006737, 72.899696, 4.996681, 72.911644


    polygon_array = [[north, west],
                      [south, west],
                        [south, east],
                          [north, east],
                            [north, west]]

    PreTimeSeries(island, 'Maldives', atoll='South Maalhosmadulu', polygon=polygon_array).main()


-------------------------------------------------------------------
Retrieving general and spatial information about the island
Island: Olhugiri (South Maalhosmadulu), Maldives
-------------------------------------------------------------------

~ The following information will be extracted/calculated: other and duvat_magnan_2019 ~
No other information available.
~ Retrieving information from Duvat & Magnan (2019). ~
Island not found in the database.


In [72]:
island_info = retrieve_island_info('Dhakandhoo', 'Maldives', verbose=False)
island_info['spatial_reference']['polygon']

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              72.89049519999999,
              5.2501492
            ],
            [
              72.89049519999999,
              5.2468208999999995
            ],
            [
              72.8968577,
              5.2468208999999995
            ],
            [
              72.8968577,
              5.2501492
            ],
            [
              72.89049519999999,
              5.2501492
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [49]:
island_info['timeseries_climate_indices']['timeseries'].columns

Index(['Dipole Mode Index (Indian Ocean Dipole Index)',
       'Pacific North American Index',
       'East Pacific/North Pacific Oscillation', 'Western Pacific Index',
       'Eastern Atlantic/Western Russia', 'North Atlantic Oscillation',
       'North Atlantic Oscillation (Jones)', 'Southern Oscillation Index',
       'Eastern Tropical Pacific SST (anomalies)',
       'Eastern Tropical Pacific SST (mean values)',
       'Bivariate ENSO Timeseries', 'Tropical Northern Atlantic Index',
       'Tropical Southern Atlantic Index', 'Western Hemisphere Warm Pool',
       'Oceanic Niño Index', 'Multivariate ENSO Index (MEI V2)',
       'Extreme Eastern Tropical Pacific SST (anomalies)',
       'Extreme Eastern Tropical Pacific SST (mean values)',
       'Central Tropical Pacific SST (anomalies)',
       'Central Tropical Pacific SST (mean values)',
       'East Central Tropical Pacific SST (anomalies)',
       'East Central Tropical Pacific SST (mean values)',
       'Pacific Decadal Oscill

In [52]:
island_info['timeseries_climate_indices']['timeseries']['Multivariate ENSO Index (MEI V2)'].plot()

<Axes: >

In [1]:
import matplotlib.pyplot as plt
import Rbeast as rb
from datetime import datetime
import os
import numpy as np
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

%load_ext autoreload
%autoreload 2

In [34]:
tags = {'place':'islet', 'natural':'coastline'}
gdf = ox.features_from_place("Faadhippolhu", tags=tags)

In [35]:
gdf['name:en'].way.values

array([nan, nan, 'Maabinhuraa', nan, 'Olhuvelifushi', nan,
       'Thilamaafushi', 'Fainuaadhamhuraa', nan, nan, nan, 'Innahura',
       nan, nan, nan, nan, 'Raiyruhhuraa', nan, 'Mayyaafushi', nan,
       'Faadhoo', nan, nan, nan, 'Meedhaahuraa', 'Kurendhoo', nan,
       'Maafilaafushi', 'Lhohi', 'Lhossalafushi', 'Aligaa', 'Medhafushi',
       'Varihuraa', nan, nan, 'Madivaru', 'Vavvaru', nan, 'Felivaru',
       'Veyvah', nan, nan, 'Naifaru', nan, nan, nan, nan, 'Musleygihuraa',
       nan, nan, 'Medhadhihuraa', nan, 'Gaavelifaru', 'Hinnavaru'],
      dtype=object)

In [ ]:
islands = gdf['name:en'].way.values
for island in islands:
    if type(island) == str:
        try:
            PreTimeSeries(island, 'Maldives').main()
        except:
            continue

In [2]:
path_to_data = os.path.join(os.getcwd(), 'data', 'info_islands')

latitude, longitude, colors, names, desc = [], [], [], [], []

for file in os.listdir(path_to_data):
    island = file.split('_')[1] 
    country = file.split('_')[2].split('.')[0]
    island_info = retrieve_island_info(island, country, verbose=False)
    latitude.append(island_info['spatial_reference']['latitude'])
    longitude.append(island_info['spatial_reference']['longitude'])
    names.append(island_info['general_info']['island'])

    if 'visual_inspection' in island_info['general_info'].keys():
        colors.append('blue')
        desc.append('Visual inspection done')
    
    else:
        if 'timeseries_coastsat' in island_info.keys():
            if 'timeseries' in island_info['timeseries_coastsat'].keys():
                colors.append('green')
                desc.append('CoastSat time series available')
            
            else:
                if os.path.exists(os.path.join(os.getcwd(), 'data', 'coastsat_data', island+'_'+country)):
                    colors.append('orange')
                    desc.append('CoastSat images are being downloaded')
        
        else:
            if os.path.exists(os.path.join(os.getcwd(), 'data', 'coastsat_data', island+'_'+country)):
                colors.append('orange')
                desc.append('CoastSat images are being downloaded')
            
            else:
                colors.append('red')
                desc.append('CoastSat has not started downloading images yet')

# Create a base map
m = folium.Map(location=[sum(latitude) / len(latitude), sum(longitude) / len(longitude)], zoom_start=5)

# Add markers for each point
for lat, lon, color, name, des in zip(latitude, longitude, colors, names, desc):
    folium.Marker([lat, lon], popup=str(name)+'\n'+str(des), icon=folium.Icon(color=color)).add_to(m)

# Save the map to an HTML file
m.save('progress_island_mapping.html')

# Open the map in a web browser
webbrowser.open('progress_island_mapping.html')

True

In [28]:
import pyproj
from bs4 import BeautifulSoup

island = 'Vodamulaa'
country = 'Maldives'

island_info = retrieve_island_info(island, country, verbose=False)
reference_shoreline = island_info['spatial_reference']['reference_shoreline']
transects = island_info['spatial_reference']['transects']
polygon_reference_shoreline = shapely.geometry.Polygon(reference_shoreline)

latitude, longitude = island_info['spatial_reference']['latitude'], island_info['spatial_reference']['longitude']
ts_analysis_results = island_info['timeseries_analysis']
key_transects = [int((key).split('_')[3]) for key in ts_analysis_results.keys()]
intersections = [polygon_reference_shoreline.exterior.intersection(shapely.geometry.LineString(transects[key_transect])) for key_transect in key_transects]
x_intersections = [intersection.x for intersection in intersections]
y_intersections = [intersection.y for intersection in intersections]

m = folium.Map(location=[latitude, longitude], zoom_start=15)

# Reproject the coordinates to the Web Mercator projection
# Source and target coordinate reference systems
tgt_crs = pyproj.CRS('EPSG:4326')
src_crs = pyproj.CRS('EPSG:3857')

# Define transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# Transform latitude and longitude to x and y
x_intersections, y_intersections = transformer.transform(x_intersections, y_intersections)

# Transform reference shoreline to x and y
x_reference_shoreline, y_reference_shoreline = transformer.transform(polygon_reference_shoreline.exterior.coords.xy[0], polygon_reference_shoreline.exterior.coords.xy[1])

c_trend = [ts_analysis_results[val]['trend']['trend_slope'] for val in ts_analysis_results.keys()]
symbols_trend = [ts_analysis_results[val]['trend']['trend_result'] for val in ts_analysis_results.keys()]

# Add markers for each point with different shapes
for lat, lon, symbol in zip(y_intersections, x_intersections, symbols_trend):
    if symbol == "increasing":
        icon = folium.Icon(color='green', icon='arrow-up')
    elif symbol == "decreasing":
        icon = folium.Icon(color='red', icon='arrow-down')
    elif symbol == "no trend":
        icon = folium.Icon(color='orange', icon='minus')
    else:
        icon = folium.Icon(color='black', icon='question-sign')

    folium.Marker(
        location=[lat, lon],
        icon=icon
    ).add_to(m)

folium.PolyLine(np.dstack((y_reference_shoreline, x_reference_shoreline)), tooltip="Coast", color='black').add_to(m)

# Save the map to an HTML file
m.save('results_islands_mapping.html')

# Open the map in a web browser
webbrowser.open('results_islands_mapping.html')

True